In [6]:
from speechbrain.pretrained import foreign_class

classifier = foreign_class(
    source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
    pymodule_file="custom_interface.py",
    classname="CustomEncoderWav2vec2Classifier",
)

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
C:\Users\chaon\Projects\emotion_recognition\venv\lib\site-packages\transformers\configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from t

# MELD data

In [15]:

import pandas as pd

df = pd.read_csv(r"C:\Users\kejiongli\Projects\emotion_recognition\backend\MELD\test_sent_emo.csv")

In [16]:
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"
...,...,...,...,...,...,...,...,...,...,...,...
2605,2760,"Yeah, I mean, come on Ross, no one will even n...",Rachel,neutral,neutral,279,11,6,4,"00:14:35,457","00:14:40,211"
2606,2761,They’re not listening too me?,Ross,surprise,negative,279,12,6,4,"00:14:42,256","00:14:43,840"
2607,2762,Of course they’re listening to you! Everybody ...,Rachel,neutral,neutral,279,13,6,4,"00:14:44,008","00:14:48,511"
2608,2763,Monica you really think I should try this phas...,Ross,neutral,neutral,279,14,6,4,"00:14:48,138","00:14:52,390"


In [32]:
from pathlib import Path

for i, f in enumerate((Path('.') / 'MELD' / 'audio_files').iterdir()):
    dia, utt = f.stem.split('_')
    dia = int(dia[3:])
    utt = int(utt[3:])
    cond = (df['Utterance_ID'] == utt) & (df['Dialogue_ID'] == dia)

    out_prob, score, index, text_lab = classifier.classify_file(str(f))

    df.loc[cond, 'SpeechBrain'] = text_lab[0]

In [33]:
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,SpeechBrain
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",ang
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",hap
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",neu
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",ang
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833",hap
...,...,...,...,...,...,...,...,...,...,...,...,...
2605,2760,"Yeah, I mean, come on Ross, no one will even n...",Rachel,neutral,neutral,279,11,6,4,"00:14:35,457","00:14:40,211",ang
2606,2761,They’re not listening too me?,Ross,surprise,negative,279,12,6,4,"00:14:42,256","00:14:43,840",ang
2607,2762,Of course they’re listening to you! Everybody ...,Rachel,neutral,neutral,279,13,6,4,"00:14:44,008","00:14:48,511",ang
2608,2763,Monica you really think I should try this phas...,Ross,neutral,neutral,279,14,6,4,"00:14:48,138","00:14:52,390",neu


In [57]:
# cond = (df.Emotion == 'disgust') & (df.SpeechBrain == 'ang')
# cond = (df.Emotion == 'joy') & (df.SpeechBrain == 'hap')
# cond = (df.Emotion == 'sadness') & (df.SpeechBrain == 'sad')
# cond = (df.Emotion == 'neutral') & (df.SpeechBrain == 'neu')
cond = (df.Emotion == 'anger') & (df.SpeechBrain == 'ang')
# cond &= (df.Speaker == 'Ross')
# cond &= (df.Speaker == 'Rachel')
# cond &= (df.Speaker == 'Joey')
cond &= (df.Speaker == 'Monica')
df[cond]

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,SpeechBrain
156,180,I know!,Monica,anger,negative,17,8,3,7,"00:02:47,250","00:02:51,420",ang
327,359,Rachel! Im never gonna think its okay for yo...,Monica,anger,negative,34,17,6,15,"00:18:57,928","00:19:01,556",ang
335,367,Behind my brother's back? ... is exactly the ...,Monica,anger,negative,36,1,1,24,"00:17:58,202","00:18:05,249",ang
396,436,Phoebe! Come on! Lets go! Come on! Why aren...,Monica,anger,negative,45,0,7,1,"00:10:07,732","00:10:15,864",ang
707,756,"No its not! You want it, youre gonna have to...",Monica,anger,negative,77,3,3,9,"00:12:20,448","00:12:23,575",ang
724,776,Rachel always cries!,Monica,anger,negative,80,3,4,4,"00:06:57,542","00:06:59,251",ang
864,921,"Mom says it's all of Manhattan, parts of Brook...",Monica,anger,negative,98,1,1,7,"00:01:46,481","00:01:51,110",ang
866,923,Pants and a sweater?,Monica,anger,negative,98,3,1,7,"0:01:53,530","0:01:54,842",ang
868,925,Who am I gonna meet in a blackout?,Monica,anger,negative,98,5,1,7,"00:01:56,074","00:01:57,825",ang
869,926,Power company guys?,Monica,anger,negative,98,6,1,7,"0:01:59,327","0:02:00,787",ang


In [41]:
df.Emotion.unique()

array(['surprise', 'anger', 'neutral', 'joy', 'sadness', 'fear',
       'disgust'], dtype=object)

In [42]:
df.SpeechBrain.unique()

array(['ang', 'hap', 'neu', 'sad'], dtype=object)

# Recorded audio

In [ ]:
from pathlib import Path

data = []

for i, f in enumerate((Path('.') / 'RecordedAudio' / 'audio_files').iterdir()):
    out_prob, score, index, text_lab = classifier.classify_file(str(f))
    data.append(dict(id=f.stem, speaker=f.stem.split('_')[0], emotion=text_lab[0]))

In [ ]:
import pandas as pd

df = pd.DataFrame(data=data)
df

In [ ]:
df.to_csv('./RecordedAudio/all.csv')